In [1]:
import pandas as pd
import math
import datetime
import urllib.request
import json
from os import listdir

In [2]:
y_key = {"key": "YQPW47F7R28CPSTSU5KF8MAPT", "num": 0}
l_key = {"key":"4YFQEHEB2CHXEK6GE35RKSRSG", "num": 0}
w_key = {"key": "2F3H4DT4UCYGK72JVWJXJXERF", "num": 0}
keys = [y_key, l_key, w_key]

In [3]:
df = pd.read_csv("data/fatal-police-shootings-data.csv")
len(df)

7291

In [4]:
len(df[df.longitude.isna()])

711

In [5]:
len(df[df.latitude.isna()])

711

In [6]:
type(df["date"][0])

str

In [7]:
def get_key():
    for key in keys:
        if key['num'] < 1000:
            key['num'] +=1
            return key['key']
    raise Exception("Sorry, out of keys")

In [8]:
###STOP
### MAKE SURE NO ONE HAS RUN THIS TODAY!

weather_api_endpoint = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline" 
for i, row in df[5000:].iterrows(): 
    
    documentNumber=row['id']
    if f'{documentNumber}.json' in listdir("data/weather_jsons"):
        continue #this will redo all the bad points, could be fixed. 
    
    weather_api_key = get_key()
    
    
    
    latitude=row['latitude']
    longitude=row['longitude']
    datetimeStr=row['date'] 

    if math.isnan(latitude) or math.isnan(longitude) or (latitude==0 and longitude==0):
        #print("Bad latlon {},{}".format(latitude,longitude))
        continue


    try:
        date_time=datetime.datetime.strptime(datetimeStr, '%Y-%m-%d')
    except ValueError:
        print("Bad date format {}".format(datetimeStr))
        continue

    latitude='{:.5f}'.format(latitude)
    longitude='{:.5f}'.format(longitude) 
   
    query = f"{weather_api_endpoint}/{latitude},{longitude}/{datetimeStr}?key={weather_api_key}&include=days"
    try:
        response = urllib.request.urlopen(query)
        data = response.read()
    except Exception:
        print("Error reading from {}".format(query))
        continue

    weatherData = json.loads(data.decode('utf-8')) 

    errorCode=weatherData["errorCode"] if 'errorCode' in weatherData else 0

    if (errorCode>0):
        print("Error reading from errorCode {}, error={}".format(query, errorCode))
        continue
    
    with open(f'data/weather_jsons/{documentNumber}.json', 'w') as outfile:
        json.dump(weatherData, outfile)


In [9]:
jsons = []
for item in listdir("data/weather_jsons"):
    if ".json" in item:
        ID = item[:-5]
        #print(item, ID)
        with open(f"data/weather_jsons/{item}","r") as f:
            data = json.load(f)
        dict1 = data["days"][0]
        if "stations" in dict1:
            del dict1["stations"]
        dict1["ID"] = ID
        jsons.append(dict1)
        

In [10]:
jsons[0]

{'datetime': '2018-05-30',
 'datetimeEpoch': 1527663600,
 'tempmax': 67.9,
 'tempmin': 57.6,
 'temp': 61.8,
 'feelslikemax': 67.9,
 'feelslikemin': 57.6,
 'feelslike': 61.8,
 'dew': 56.3,
 'humidity': 82.8,
 'precip': 0.02,
 'precipprob': 100.0,
 'precipcover': 8.33,
 'preciptype': ['rain'],
 'snow': 0.0,
 'snowdepth': 0.0,
 'windgust': None,
 'windspeed': 9.1,
 'winddir': 165.7,
 'pressure': 1010.4,
 'cloudcover': 98.6,
 'visibility': 8.3,
 'solarradiation': 67.9,
 'solarenergy': 6.0,
 'uvindex': 3.0,
 'sunrise': '05:43:30',
 'sunriseEpoch': 1527684210,
 'sunset': '19:59:24',
 'sunsetEpoch': 1527735564,
 'moonphase': 0.51,
 'conditions': 'Rain, Overcast',
 'description': 'Cloudy skies throughout the day with morning rain.',
 'icon': 'rain',
 'source': 'obs',
 'ID': '3721'}

In [11]:
weatherData["days"][0]

{'datetime': '2022-04-20',
 'datetimeEpoch': 1650430800,
 'tempmax': 36.4,
 'tempmin': 29.8,
 'temp': 33.1,
 'feelslikemax': 32.3,
 'feelslikemin': 20.3,
 'feelslike': 26.5,
 'dew': 26.0,
 'humidity': 76.7,
 'precip': 0.28,
 'precipprob': 100.0,
 'precipcover': 20.83,
 'preciptype': ['snow'],
 'snow': 0.8,
 'snowdepth': 4.3,
 'windgust': 34.7,
 'windspeed': 14.9,
 'winddir': 155.7,
 'pressure': 1011.9,
 'cloudcover': 97.8,
 'visibility': 6.9,
 'solarradiation': 35.8,
 'solarenergy': 3.3,
 'uvindex': 1.0,
 'severerisk': 10.0,
 'sunrise': '06:13:28',
 'sunriseEpoch': 1650453208,
 'sunset': '20:08:16',
 'sunsetEpoch': 1650503296,
 'moonphase': 0.59,
 'conditions': 'Snow, Overcast',
 'description': 'Cloudy skies throughout the day with snow.',
 'icon': 'snow',
 'stations': ['HIBM5',
  'KEVM',
  '72654954905',
  'KHIB',
  '72745594931',
  'KCQM',
  '72747404923'],
 'source': 'obs'}

In [16]:
pd.DataFrame(jsons).to_csv("data/final_weather.csv")

In [76]:
df

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,-122.892,45.487,True
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,-97.281,37.695,True
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,-122.422,37.763,True
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,-104.692,40.384,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,7946,NaN,2022-04-19,shot,gun,NaN,M,NaN,Pasadena,TX,False,attack,NaN,False,-95.196,29.664,True
7287,7949,Chris Honea,2022-04-19,shot,undetermined,26.0,M,NaN,Gordon County,GA,False,other,Car,False,-83.745,32.672,True
7288,7944,NaN,2022-04-20,shot,gun,NaN,M,H,Bakersfield,CA,False,attack,Other,False,-119.020,35.324,True
7289,7950,NaN,2022-04-20,shot,stake,NaN,M,NaN,Washington,DC,False,other,NaN,False,-77.061,38.955,True


In [6]:
#Combine the weather data and the raw data.
import pandas as pd
main_df = pd.read_csv("data/fatal-police-shootings-data.csv")
weather_df = pd.read_csv("data/final_weather.csv")
main_df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact,smha_exp_per_capita,smi_prevalance
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True,128.46,0.0472
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,-122.892,45.487,True,234.87,0.0483
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,-97.281,37.695,True,124.11,0.0412
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,-122.422,37.763,True,174.98,0.0379
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,-104.692,40.384,True,121.90,0.0390


In [7]:
weather_df.head()

,Unnamed: 0,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,source,ID,severerisk
0,0,2018-05-30,1527663600,67.9,57.6,61.8,67.9,57.6,61.8,56.3,...,1527684210,19:59:24,1527735564,0.51,"Rain, Overcast",Cloudy skies throughout the day with morning r...,rain,obs,3721,NaN
1,1,2017-08-02,1501657200,94.5,56.9,73.6,90.9,56.9,72.8,53.5,...,1501679609,20:17:00,1501730220,0.38,Clear,Clear conditions throughout the day.,clear-day,obs,2833,NaN
2,2,2015-08-12,1439355600,100.2,73.1,87.0,97.8,73.1,85.6,53.6,...,1439381528,20:36:32,1439429792,0.98,Clear,Clear conditions throughout the day.,clear-day,obs,729,NaN
3,3,2021-01-14,1610604000,61.2,40.0,49.6,61.2,33.4,46.9,35.0,...,1610630438,17:38:18,1610667498,0.01,Clear,Clear conditions throughout the day.,clear-day,obs,6523,NaN
4,4,2021-01-09,1610168400,50.7,32.0,40.2,50.7,28.8,36.7,31.4,...,1610195069,17:23:37,1610231017,0.92,"Snow, Rain",Clear conditions throughout the day with morni...,rain,obs,6489,NaN


In [9]:
df = pd.merge(main_df, weather_df, left_on="id",right_on= "ID")

In [11]:
print(len(main_df))
print(len(weather_df))
df

7291
6579


,id,name,date,manner_of_death,armed,age,gender,race,city,state,...,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,source,ID,severerisk
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,...,1420214361,16:33:56,1420245236,0.46,"Rain, Partially cloudy",Partly cloudy throughout the day with afternoo...,rain,obs,3,NaN
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,...,1420213904,16:39:43,1420245583,0.46,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,obs,4,NaN
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,...,1420292681,17:22:43,1420327363,0.49,"Snow, Rain, Partially cloudy",Partly cloudy throughout the day with rain or ...,rain,obs,5,NaN
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,...,1420385130,17:04:00,1420419840,0.50,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,obs,8,NaN
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,...,1420381325,16:45:33,1420415133,0.50,Clear,Clear conditions throughout the day.,clear-day,obs,9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6574,7946,NaN,2022-04-19,shot,gun,NaN,M,NaN,Pasadena,TX,...,1650368993,19:50:19,1650415819,0.55,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,obs,7946,10.0
6575,7949,Chris Honea,2022-04-19,shot,undetermined,26.0,M,NaN,Gordon County,GA,...,1650366041,20:07:59,1650413279,0.55,Clear,Clear conditions throughout the day.,clear-day,obs,7949,10.0
6576,7944,NaN,2022-04-20,shot,gun,NaN,M,H,Bakersfield,CA,...,1650460631,19:33:17,1650508397,0.59,Partially cloudy,Clearing in the afternoon.,partly-cloudy-day,obs,7944,10.0
6577,7950,NaN,2022-04-20,shot,stake,NaN,M,NaN,Washington,DC,...,1650450272,19:50:24,1650498624,0.59,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,obs,7950,10.0


In [14]:
import numpy as np


,id,name,date,manner_of_death,armed,age,gender,race,city,state,...,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,source,ID,severerisk


In [17]:
df.to_csv("data/weather&policeshootings.csv")